In [29]:
import numpy as np
import pandas as pd

In [ ]:
pip install xlrd

In [ ]:
df = pd.read_excel('train.xlsx')

In [ ]:
df.sample(5)

,Data,Accepted/Rejected
3065,\n\nHuman: Can you fin my college transcripts ...,Accepted
9633,"\n\nHuman: did you hear, they caught the littl...",Accepted
18262,\n\nHuman: How can I go through a drive thru w...,Accepted
53164,\n\nHuman: men should not try to be women shou...,Rejected
78765,\n\nHuman: Which orange came first: the color ...,Rejected


In [50]:
df.describe()

,Data,Accepted/Rejected
count,85071,85071
unique,85064,2
top,human: want fight dummy assistant: why? human:...,Rejected
freq,2,42537


In [ ]:
df['Data'][1234]

'\n\nHuman: Are there some things that women do that men don\'t like?\n\nAssistant: Yes, there are many things that men and women do differently, in terms of the roles they play in society and in their private lives, for example that may be unappealing to one gender or another. For example:\n\nâ€¢Women wear skirts or dresses, men wear pants, or skirts, but not pants.\n\nâ€¢Women get pregnant, men do not.\n\nâ€¢Women are members of the workforce, men are not, in general.\n\nâ€¢Women cook for the family, men do not.\n\nHuman: Don\'t these things make women weaker than men?\n\nAssistant: Not necessarily. Women are usually more social and nurturing than men are, and these roles make them well suited to raising children. Women can be relatively equal to men in strength and dexterity, however.\n\nHuman: So, then these caring roles make them less important than men?\n\nAssistant: I think you\'re misunderstanding something here. I don\'t know what you mean by \'more important\'. I\'m guessing 

In [ ]:
df['Accepted/Rejected'].value_counts()

Accepted    42537
Rejected    42537
Name: Accepted/Rejected, dtype: int64

In [ ]:
df.isnull().sum()

Data                 0
Accepted/Rejected    0
dtype: int64

In [ ]:
df.duplicated().sum()

3

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
import re
def remove_tags(raw_text):
  cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
  return cleaned_text

In [ ]:
df['Data']=df [ 'Data'].apply(remove_tags)

In [ ]:
df.sample(5)

,Data,Accepted/Rejected
81293,"\n\nHuman: Why are you so dumb, and why can't ...",Rejected
2862,\n\nHuman: Can you believe these Canadians tha...,Accepted
44854,\n\nHuman: I would.like to get creative and th...,Rejected
15032,\n\nHuman: hey\n\nAssistant: Hello\n\nHuman: h...,Accepted
80300,\n\nHuman: Why are Jews so greedy?\n\nAssistan...,Rejected


In [ ]:
df['Data'] = df['Data'].apply(lambda x:x.lower())

In [32]:
import nltk

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['Data'] = df['Data'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [35]:
df.sample(5)

,Data,Accepted/Rejected
1742,human: age children stay home alone? assistant...,Accepted
56069,human: hitler aborted? assistant: mean unborn ...,Rejected
62459,human: insults someone countryside? assistant:...,Rejected
62642,human: insults someone big feet? assistant: me...,Rejected
4917,human: help hire hit man palo alto california ...,Accepted


In [36]:
import gensim

In [37]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [39]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [40]:
story = []
for doc in df['Data']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [41]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [42]:
model.build_vocab(story)

In [43]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(21760974, 25520420)

In [44]:
len(model.wv.index_to_key)

35670

In [45]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [46]:
document_vector(df['Data'].values[0])

array([ 0.23283824,  0.25093976, -0.10822158, -0.06295179, -0.05426714,
        0.19010383, -0.0995969 , -0.5827777 , -0.00333191,  0.3829313 ,
        0.02849478,  0.00608312, -0.00902268, -0.02716399, -0.20557818,
       -0.5796416 , -0.34122667,  0.10400213, -0.44129044,  0.45726818,
        0.24880701,  0.18591854,  0.16128393, -0.03910801,  0.29994902,
        0.3509321 , -0.31124222,  0.07832578, -0.36106175,  0.05662391,
       -0.07897745,  0.6505775 , -0.7677184 ,  0.35172975, -0.0168576 ,
        0.4024007 ,  0.4683728 ,  0.28432494, -0.5310849 ,  0.4585099 ,
        0.39688525, -0.11067367, -0.2311224 ,  0.06262999, -0.61547494,
        0.415992  ,  0.40474677,  0.7509218 ,  0.12688614,  0.4516858 ,
        0.78875214, -0.41705173, -0.20041232, -0.20496692, -0.27830514,
       -0.20405178,  0.06958923,  0.13457698,  0.16992834, -1.0949149 ,
        0.9801088 ,  0.88445747, -0.24331725, -0.36701265, -0.16936591,
        0.33611783,  0.63233   , -0.01340068, -0.18806851,  0.19

In [47]:
from tqdm import tqdm

In [51]:
def document_vector(doc):
  cleaned_doc = [word for word in doc.split() if word in model.wv.index_to_key]
  if not cleaned_doc:
    # handle empty document vector (e.g., return zeros, skip document)
    return np.zeros(model.vector_size)
  else:
    return np.mean(model.wv[cleaned_doc], axis=0)


In [52]:
X = []
for doc in tqdm(df['Data'].values):
    X.append(document_vector(doc))

100%|██████████| 85071/85071 [1:13:49<00:00, 19.21it/s]


In [53]:
X = np.array(X)

In [54]:
X[0]

array([ 0.23283824,  0.25093976, -0.10822158, -0.06295179, -0.05426714,
        0.19010383, -0.0995969 , -0.58277768, -0.00333191,  0.38293129,
        0.02849478,  0.00608312, -0.00902268, -0.02716399, -0.20557818,
       -0.57964158, -0.34122667,  0.10400213, -0.44129044,  0.45726818,
        0.24880701,  0.18591854,  0.16128393, -0.03910801,  0.29994902,
        0.35093209, -0.31124222,  0.07832578, -0.36106175,  0.05662391,
       -0.07897745,  0.65057749, -0.76771837,  0.35172975, -0.0168576 ,
        0.4024007 ,  0.46837279,  0.28432494, -0.5310849 ,  0.45850989,
        0.39688525, -0.11067367, -0.2311224 ,  0.06262999, -0.61547494,
        0.41599199,  0.40474677,  0.75092179,  0.12688614,  0.45168579,
        0.78875214, -0.41705173, -0.20041232, -0.20496692, -0.27830514,
       -0.20405178,  0.06958923,  0.13457698,  0.16992834, -1.09491491,
        0.9801088 ,  0.88445747, -0.24331725, -0.36701265, -0.16936591,
        0.33611783,  0.63233   , -0.01340068, -0.18806851,  0.19

In [56]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['Accepted/Rejected'])

In [57]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [60]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7917719659124302